In [1]:
from  test_model import do_test
from sequence_model import CTPAModel  
from sequence_dataset import ChunkDataset
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd

In [2]:
import pickle
import pandas as pd
#reading embeddings with and without offset
file_paths0 =  "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_0\\embs_test.pickle"

file_paths1 = "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_1\\embs_test.pickle"

def return_loaded_files(file_path):
  
  
    with open(file_path, 'rb') as f:
            # Load the pickled object from the file
            loaded_data_test = pickle.load(f)
    return loaded_data_test


In [3]:
emb_test = return_loaded_files(file_paths0)
emb_test1 = return_loaded_files(file_paths1)
df_test = pd.DataFrame(emb_test)
df_test["emb1"] = emb_test1["emb"]
df_test["prob1"] = emb_test1["prob"]
test_dataset = ChunkDataset(df_test)
test_dataloader = DataLoader(test_dataset, batch_size=2, num_workers=0,drop_last=True)


In [4]:
embed_dim = 384
num_heads = 8
num_layers = 4

model = CTPAModel(embed_dim, num_heads, num_layers, drop_prob=0.1)
model.to("cuda")

CTPAModel(
  (pos_embedding): Embedding(1024, 384)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
    )
    (linear1): Linear(in_features=384, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=384, bias=True)
    (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
        )
        (linear1): Linear(in_features=384, out_features=2048, bias=True)
        (dropout): Dropout(p

In [5]:
state_dict = torch.load(r"C:\Users\preet\Documents\penet\results_sequence\best_model.pth") 
model.load_state_dict(state_dict)
metrics, output = do_test(test_dataloader, model)
del metrics["Loss"]
print(metrics)
metrics_df = pd.DataFrame.from_dict([metrics])
print(metrics_df.head())

  0%|                                                                                           | 0/81 [00:00<?, ?it/s]C:\Users\preet\Documents\penet\sequence_model\sequence_dataset.py:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  x =torch.tensor(emblist)
C:\Users\preet\Documents\penet\sequence_model\test_model.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output.extend(F.softmax(outputs).cpu().numpy()[:, 1])
100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 89.50it/s]

Test loss0.516, F1 0.733, Acc 0.796, BA 0.834, precision 0.455, recall 0.893
cm[[104  30]
 [  3  25]]
{'Accuracy': 0.7962962962962963, 'Sensitivity': 0.8928571428571429, 'Specificity': 0.7761194029850746, 'Precision': 0.45454545454545453, 'AUC-ROC': 0.9442963752665245, 'AUC-PR': 0.7829526731862685, 'Balanced Accuracy': 0.8344882729211087, 'F1 Score': 0.732740088986652}
   Accuracy  Sensitivity  Specificity  Precision   AUC-ROC    AUC-PR  \
0  0.796296     0.892857     0.776119   0.454545  0.944296  0.782953   

   Balanced Accuracy  F1 Score  
0           0.834488   0.73274  
